In [2]:
import pandas as pd
import numpy as np

# Carica i dataset
dataset_x = [pd.read_csv(f'dataset/{x}.csv') for x in range(2009, 2019)]
dataset_y =  pd.read_csv('dataset/flights.csv')

In [5]:
final_data = []
proportion = 0.001

for df in dataset_x:

    # Unisci df con dataset_y usando tutte le colonne come chiavi e seleziona solo le righe che non hanno corrispondenza
    merged = df.merge(dataset_y, how='left', indicator=True)
    df = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
    del merged

    df.drop_duplicates(inplace=True)
    grouped = df.groupby('FL_DATE')
    
    selected_rows = []
    for _, day_group in grouped:
        day_selected = []
        
        # Raggruppa per 'OP_CARRIER' all'interno di ogni giorno
        carrier_grouped = day_group.groupby('OP_CARRIER')
        
        for _, carrier_group in carrier_grouped:
            proportionate_sample_size = int(len(carrier_group) * proportion)
            
            if(proportionate_sample_size > 0):
                sampled = carrier_group.sample(n=proportionate_sample_size, replace=False, random_state=np.random.randint(0, 1000))
                day_selected.append(sampled)
            else:
                break
        
        if(len(day_selected) == len(carrier_grouped)): 
            day_selected_data = pd.concat(day_selected)
            selected_rows.append(day_selected_data)
        else:
            proportionate_sample_size = int(len(day_group) * proportion)
            replace = None
            # Verifica se ci sono abbastanza elementi per un campionamento senza ripetizione
            if len(day_group) < proportionate_sample_size:
                replace = True  # Usare il campionamento con ripetizione
            else:
                replace = False  # Usare il campionamento senza ripetizione

            # Esegui il campionamento
            sampled = day_group.sample(n=proportionate_sample_size, replace=replace, random_state=np.random.randint(0, 1000))
            day_selected_data = pd.concat([sampled])
            selected_rows.append(day_selected_data)
    
    # Combina i campioni di ogni giorno
    selected_data = pd.concat(selected_rows)
    
    final_data.append(selected_data)

# Combina tutti i dati selezionati
final_combined_data = pd.concat(final_data, ignore_index=True)
final_combined_data.to_csv('dataset/test_flights.csv', index=False)